# Подготовка среды

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import shutil
# shutil.copytree("/content/drive/Shared drives/hse_BERT/hse_Af_Tr_BERT/embeddings/bert-base-multilingual-cased-polit3/bin", "/content/data/bin2")

'/content/data/bin2'

In [2]:
import os
os.chdir('/content/drive/Shared drives/hse_BERT/hse_Af_Tr_BERT')

In [3]:
folder = "bert-base-multilingual-cased-polit"

if not os.path.isdir(os.path.join("embeddings", folder)):
    os.mkdir(os.path.join("embeddings", folder))
if not os.path.isdir(os.path.join("embeddings", folder, "bin")):
    os.mkdir(os.path.join("embeddings", folder, "bin"))
if not os.path.isdir(os.path.join("embeddings", folder, "ready")):
    os.mkdir(os.path.join("embeddings", folder, "ready"))

In [4]:
!pip install transformers
!pip install razdel

     |████████████████████████████████| 2.1MB 4.8MB/s 
     |████████████████████████████████| 3.3MB 25.5MB/s 
     |████████████████████████████████| 901kB 44.4MB/s 


In [5]:
import torch
import nltk
import re
import json
import time
import numpy as np
import pandas as pd
import pickle as pc
from transformers import BertTokenizer, BertModel

import matplotlib.pyplot as plt
% matplotlib inline

from scipy.spatial.distance import cosine
from razdel import sentenize

In [6]:
def my_dump(file, name):
    with open("./" + name + ".pic", 'wb') as f:
        pc.dump(file, f)

def my_load(name):
    with open("./" + name + ".pic", 'rb') as f:
        return pc.load(f)

In [ ]:
# Load pre-trained model tokenizer (vocabulary)
# 'DeepPavlov/rubert-base-cased-sentence'
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [ ]:
model = BertModel.from_pretrained('bert-base-multilingual-cased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )
model.eval()

#text processing

## generate (split) sentences

In [ ]:
%%time
# name = "all_cyberleninka_polit2"
name = "all_cyberleninka_liter2"
with open("./texts/{}/{}.txt".format(name, name), 'r') as f:
    data = f.read()
sent2 = list(sentenize(data))
my_dump(sent2, name)

CPU times: user 6min 1s, sys: 12.6 s, total: 6min 14s
Wall time: 6min 48s


#getting embeddings

## read sentences

In [ ]:
%%time
name = "bin/all_cyberleninka_polit2"
# name = "all_cyberleninka_liter2"
sent1 = my_load(name)

CPU times: user 8.68 s, sys: 2.41 s, total: 11.1 s
Wall time: 14.1 s


In [ ]:
def get_text_from_ST(ss):
    return ss.text
sent = list(map(get_text_from_ST, sent1))
len(sent)

2348859

In [ ]:
del sent1

##process

In [ ]:
def sum_tensors(token):
    return torch.sum(token[-4:], dim=0)

def get_embeddings(text):
    tokenized_text = tokenizer.tokenize("[CLS] " + text.lower() + " [SEP]")
    if len(tokenized_text) > 80:
        return [],[]
    marked_text = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)

    tokens_tensor = torch.tensor([marked_text])
    segments_tensors = torch.tensor([segments_ids])

    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2]

    token_embeddings = torch.squeeze(torch.stack(hidden_states, dim=0),
                                    dim=1).permute(1,0,2)
    
    token_vecs_sum = list(map(sum_tensors, token_embeddings))
    return tokenized_text, token_vecs_sum

# token_vecs = hidden_states[-2][0]
# sentence_embedding = torch.mean(token_vecs, dim=0)

In [ ]:
# text = "Тюмень стала самым «счастливым» городом России по уровню довольных своим городом людей."

def sum_tensors(token):
    return torch.sum(token[-4:], dim=0)

def get_embeddings2(text):
    tokenized_text = tokenizer.tokenize("[CLS] " + text.lower() + " [SEP]")
    if len(tokenized_text) > 80:
        return [],[]
    marked_text = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)

    tokens_tensor = torch.tensor([marked_text])
    segments_tensors = torch.tensor([segments_ids])

    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2]

    token_embeddings = torch.squeeze(torch.stack(hidden_states, dim=0),
                                    dim=1).permute(1,0,2)

    token_vecs_sum = list(map(sum_tensors, token_embeddings))

    new_tokens = []
    new_token_vecs = []
    j = -1
    for i in range(len(tokenized_text)):
        if len(tokenized_text[i]) > 2 and tokenized_text[i][0] == "#" and tokenized_text[i][1] == "#":
            new_tokens[j] = new_tokens[j] + tokenized_text[i][2:]
            new_token_vecs[j].append(token_vecs_sum[i])
        else:
            new_tokens.append(tokenized_text[i])
            new_token_vecs.append([token_vecs_sum[i]])
            j+=1
    new_token_vecs2 = []
    for i in new_token_vecs:
        new_token_vecs2.append(sum(i))
    return new_tokens, new_token_vecs2

In [ ]:
def get_batch_number(from_i = 0, to_i = 1000):
    
    files_r = os.listdir("./embeddings/{}/bin".format(folder))
    cut_files_r = []
    for i in files_r:
        if i.split('.')[-1] == 'pic':
            cut_files_r.append(int(i.split('.')[0].split('_')[1].split(" ")[0]))
    diff = set(range(from_i, to_i)) - set(cut_files_r)
    return sorted(list(diff))

def reserve_batch(number):
    print("reserve {}".format(number))
    my_dump("file", "./embeddings/{}/bin/{}_{:05d}_empty".format(folder,batch, number))

batch = 1000
from_i = 0
to_i = 1000
start_time = time.time()

l = get_batch_number( from_i, to_i)
while len(l) > 0:
    j = l[0]
    reserve_batch(j)
    pr_tr = {}
    temp_time = time.time()
    for i in range(j * batch, (j + 1) * batch):
        tt, tvs = get_embeddings2(sent[i])
        for i in range(len(tt)):
            if tt[i].isalpha():# and len(tt[i]) > 2 :
                if tt[i] in pr_tr:
                    pr_tr[tt[i]].append(tvs[i])
                else:
                    pr_tr[tt[i]] = [tvs[i]]

    with open("./embeddings/{}/bin/{}_{:05d}.pic".format(folder, batch, j), 'wb') as f:
        pc.dump(pr_tr, f)
    # res.update(pr_tr)

    t = time.time()
    print("done {:05d} --- time: {}\t--- total: {}".format(j, t - temp_time, t - start_time))

    l = get_batch_number( from_i, to_i)

reserve 0
done 00000 --- time: 121.28795623779297	--- total: 121.294442653656
reserve 1
done 00001 --- time: 128.21727395057678	--- total: 249.533100605011
reserve 3
done 00003 --- time: 123.90330481529236	--- total: 373.45488572120667
reserve 4
done 00004 --- time: 128.86864304542542	--- total: 502.3444004058838
reserve 5
done 00005 --- time: 128.40703678131104	--- total: 630.7880384922028
reserve 7
done 00007 --- time: 125.1018397808075	--- total: 755.915700674057
reserve 2
done 00002 --- time: 124.66505241394043	--- total: 880.6055228710175
reserve 8
done 00008 --- time: 133.95430731773376	--- total: 1014.5854098796844
reserve 11
done 00011 --- time: 135.4564299583435	--- total: 1150.0769081115723
reserve 13
done 00013 --- time: 127.36391139030457	--- total: 1277.4652209281921
reserve 16
done 00016 --- time: 123.87614846229553	--- total: 1401.3642587661743
reserve 18
done 00018 --- time: 126.38551425933838	--- total: 1527.7762649059296
reserve 21
done 00021 --- time: 132.32456016540

In [ ]:
# %%time
# batch = 1000
# start_time = time.time()

# ftom_b = 348
# to_b = 500

# print("senteces from {} to {}".format(ftom_b * batch, to_b * batch))

# for j in range(ftom_b, to_b):
#     pr_tr = {}
#     temp_time = time.time()
#     for i in range(j * batch, (j + 1) * batch):
#         tt, tvs = get_embeddings2(sent[i])
#         for i in range(len(tt)):
#             if tt[i].isalpha() and len(tt[i]) > 2 :
#                 if tt[i] in pr_tr:
#                     pr_tr[tt[i]].append(tvs[i])
#                 else:
#                     pr_tr[tt[i]] = [tvs[i]]

#     with open("./embeddings/{}/bin/{}_{:05d}.pic".format(folder, batch, j), 'wb') as f:
#         pc.dump(pr_tr, f)

#     t = time.time()
#     print("done {:05d} --- time: {}\t--- total: {}".format(j, t - temp_time, t - start_time))

# # for i in pr_tr:
# #     print(i, len(pr_tr[i]))

#aggregation of embeddings M



In [ ]:
%%time
count_t = {}
sem_emb = {}

for name in cut_files:
    with open("./embeddings/{}/bin/{}".format(folder, name), 'rb') as f:
        data = pc.load(f)
    for i in data.keys():
        if i not in count_t:
            count_t[i] = 0
            sem_emb[i] = data[i]
        else:
            count_t[i] += 1
            sem_emb[i] = sem_emb[i] + data[i]

res = {}
for key in count_t:
    res[key] = sem_emb[key] / count_t[key]
    # print(name)

In [ ]:
with open("./embeddings/{}/dict_{}.pic".format(folder, folder), "bw") as f:
    pc.dump(res, f)

#aggregation of embeddings



In [ ]:
# folder = "bert-base-multilingual-cased-polit"

##list of files

In [ ]:
files = os.listdir("./embeddings/{}/bin".format(folder))
trash_files = []
for i in files:
    if i.split('.')[-1] == 'pic':
        if len(i.split('.')[0].split("_")) == 3:
            trash_files.append(i)

for i in files:
    if i.split('.')[-1] == 'pic':
        if len(i.split(' ') )== 2:
            trash_files.append(i)
len(trash_files)

0

In [ ]:
for file in trash_files:
    os.remove("./embeddings/{}/bin/{}".format(folder,file))
    print("./embeddings/{}/bin/{}\t removed!".format(folder,file))

./embeddings/bert-base-multilingual-cased-polit3/bin/1000_00413 (1).pic	 removed!


In [ ]:
files = os.listdir("./embeddings/{}/bin".format(folder))
cut_files = []
for i in files:
    if i.split('.')[-1] == 'pic':
        cut_files.append(i)

In [ ]:
len(cut_files)

1000

##get count

In [ ]:
%%time
from_files = {}
all_count = {}

for name in cut_files:
    with open("/content/data/bin2/{}".format( name), 'rb') as f:
        data = pc.load(f)
    for i in data.keys():
        if i not in all_count:
            all_count[i] = len(data[i])
            from_files[i] = [name]
        else:
            all_count[i] += len(data[i])
            from_files[i].append(name)
    # print(name)


CPU times: user 14min 47s, sys: 46 s, total: 15min 33s
Wall time: 15min 43s


In [ ]:
my_dump(from_files, "embeddings/{}/from_files".format(folder))
my_dump(all_count, "embeddings/{}/all_count".format(folder))

## aggregate

In [ ]:
from_files = my_load("embeddings/{}/from_files".format(folder))
all_count = my_load("embeddings/{}/all_count".format(folder))

In [ ]:
out = list(zip(all_count.keys(), list(all_count.values())))
out.sort(key = lambda i: i[1], reverse=True)
# out[:20]

In [ ]:
keys = list(all_count.keys())
k1 = []
k2 = []
k3 = []
kb = []
l = 0
for i in keys:
    l = len(i)
    if l == 1:
        k1.append(i)
    elif l == 2:
        k2.append(i)
    elif l == 3:
        k3.append(i)
    else:
        kb.append(i)
len(k1),len(k2),len(k3),len(kb)

(82, 1455, 7702, 336472)

In [ ]:
all_words = []
for i in kb:
    if all_count[i] > 10:
        all_words.append(i)
len(all_words)

54176

In [ ]:
def check_existing_emb(words):
    d = dict(out)
    fl = True
    for i in words:
        if i not in d:
            print("{} not in dictionary".format(i))
            fl = False
    return fl

def get_all_embeddings(words):
    all_files = []
    embeddings = {}
    for w in words:
        embeddings[w] = []

    for i in words:
        all_files += from_files[i]

    for name in set(all_files):
        with open("/content/data/bin2/{}".format( name), 'rb') as f:
            data = pc.load(f)
        for w in words:
            if w in data:
                embeddings[w] += data[w]
    return embeddings

def sum_embeddings(embeddings):
    res = {}
    res_emb = {}
    for i in embeddings:
        res[i] = torch.empty(len(embeddings[i]), embeddings[i][0].size(0))
        for j in range(len(embeddings[i])):
            res[i][j] = embeddings[i][j]
        res_emb[i] = res[i].mean(dim = 0)
    return res_emb

In [ ]:
# batch = 250
# st_time = time.time()

# ftom_b = 0
# to_b = 15

# print("words from {} to {}, max batchs = {}".format(ftom_b * batch, to_b * batch, len(all_words)//batch))

# all_emb = {}
# for b in range(ftom_b, to_b):
#     t_time = time.time()
#     words = all_words[b*batch:(b+1)*batch]
#     emb = get_all_embeddings(words)
#     res_emb = sum_embeddings(emb)
    
#     with open("./embeddings/{}/ready/{}_{:05d}.pic".format(folder,batch, b), 'wb') as f:
#         pc.dump(res_emb, f)

#     print("done: {} --- time: {}\t--- total: {}".format(b,time.time() - t_time, time.time() - st_time))

In [ ]:
def get_batch_number(max_i, from_i = 0):
    
    files_r = os.listdir("./embeddings/{}/ready".format(folder))
    cut_files_r = []
    for i in files_r:
        if i.split('.')[-1] == 'pic':
            cut_files_r.append(int(i.split('.')[0].split('_')[1]))
    diff = set(range(from_i, max_i)) - set(cut_files_r)
    return sorted(list(diff))

def reserve_batch(number):
    print("reserve {}".format(b))
    my_dump("file", "./embeddings/{}/ready/{}_{:05d}_empty".format(folder,batch, b))

batch = 1000
from_i = 0
st_time = time.time()
print(" max batchs = {}".format( len(all_words)//batch))

all_emb = {}
l = get_batch_number(len(all_words)//batch + 1, from_i)
while len(l) > 0:
    b = l[0]
    reserve_batch(b)
    t_time = time.time()
    words = all_words[b*batch:(b+1)*batch]
    emb = get_all_embeddings(words)
    res_emb = sum_embeddings(emb)
    
    with open("./embeddings/{}/ready/{}_{:05d}.pic".format(folder,batch, b), 'wb') as f:
        pc.dump(res_emb, f)

    print("done: {} --- time: {}\t--- total: {}".format(b,time.time() - t_time, time.time() - st_time))
    l = get_batch_number(len(all_words)//batch + 1, from_i)

 max batchs = 54
reserve 0


## delete trash

In [ ]:
files = os.listdir("./embeddings/{}/ready".format(folder))
trash_files = []
for i in files:
    if i.split('.')[-1] == 'pic':
        if len(i.split('.')[0].split("_")) == 3:
            trash_files.append(i)

In [ ]:
len(trash_files)

55

In [ ]:
for file in trash_files:
    os.remove("./embeddings/{}/ready/{}".format(folder,file))
    print("./embeddings/{}/ready/{}\t removed!".format(folder,file))

./embeddings/bert-base-multilingual-cased-polit3/ready/1000_00001_empty.pic	 removed!
./embeddings/bert-base-multilingual-cased-polit3/ready/1000_00003_empty.pic	 removed!
./embeddings/bert-base-multilingual-cased-polit3/ready/1000_00053_empty.pic	 removed!
./embeddings/bert-base-multilingual-cased-polit3/ready/1000_00004_empty.pic	 removed!
./embeddings/bert-base-multilingual-cased-polit3/ready/1000_00054_empty.pic	 removed!
./embeddings/bert-base-multilingual-cased-polit3/ready/1000_00052_empty.pic	 removed!
./embeddings/bert-base-multilingual-cased-polit3/ready/1000_00050_empty.pic	 removed!
./embeddings/bert-base-multilingual-cased-polit3/ready/1000_00002_empty.pic	 removed!
./embeddings/bert-base-multilingual-cased-polit3/ready/1000_00051_empty.pic	 removed!
./embeddings/bert-base-multilingual-cased-polit3/ready/1000_00006_empty.pic	 removed!
./embeddings/bert-base-multilingual-cased-polit3/ready/1000_00007_empty.pic	 removed!
./embeddings/bert-base-multilingual-cased-polit3/ready

In [ ]:
set(list(range(63))) - set(list(map(lambda x: int(x.split(".")[0].split("_")[1]), files)))

{55, 56, 57, 58, 59, 60, 61, 62}

#building dictionary


In [ ]:
# folder = "bert-base-multilingual-cased-liter"

In [7]:
files = os.listdir("./embeddings/{}/ready".format(folder))
res = dict()
for i in files:
    with open("./embeddings/{}/ready/{}".format(folder,i), "rb") as f:
        temp = pc.load(f)
        # print(type(temp))
        res.update(temp)

In [8]:
with open("./embeddings/{}/dict_{}.pic".format(folder, folder), "bw") as f:
    pc.dump(res, f)

In [9]:
# with open("./embeddings/{}/dict_{}.json".format(folder, folder), 'w', encoding="utf-8") as f:
#     json.dump(lista_items, f, indent=4)

#Tests

##preparation

In [10]:
# folder = "bert-base-multilingual-cased-liter"

In [11]:
with open("./embeddings/{}/dict_{}.pic".format(folder, folder), "br") as f:
    dct1  = pc.load(f)

# with open("/content/drive/Shareddrives/hse_BERT/hse_Af_Tr_BERT/embeddings/sent1/all_emc.pic", "br") as f:
#     dct1  = pc.load(f)

In [12]:
def get_word(emb, dct, nin = []):
    word = "none"
    cs = -100 
    for key in dct:
        temp_cs = 1 - cosine(emb, dct[key])
        if temp_cs > cs and key not in nin:
            cs = temp_cs
            word = key
    return word, cs

def get_word_m(emb, dct, count = 3):
    res = {}
    for key in dct:
        res[key] = 1 - cosine(emb, dct[key])
    l = list(res.items()) 
    l.sort(key= lambda x:x[1], reverse = True)
    return dict(l[:count])

def add_names(df):
    names = {': capital-common-countries' : "Capitals-common-countries",
 ': capital-world': "Capital-countries",
 ': currency': "Currency",
#  ': city-in-state',
 ': family': "Family",
 ': gram1-adjective-to-adverb': "Adjective adverb",
 ': gram2-opposite': "Opposite",
 ': gram3-comparative': "Comparative",
#  ': gram4-superlative',
#  ': gram5-present-participle',
 ': gram6-nationality-adjective': "Nationality adjective",
#  ': gram7-past-tense',
#  ': gram8-plural',
#  ': gram9-plural-verbs'
    }
    df["type"] = "none"
    for i in names:
        if i in df.index:
            df.loc[i, "type"] = names[i]

    return df

def add_names_rus(df):
    names = {': capital-common-countries' : "Известные столицы - страны",
 ': capital-world': "Столицы - страны",
 ': currency': "Валюта",
#  ': city-in-state',
 ': family': "Семья",
 ': gram1-adjective-to-adverb': "Прилагательное - наречие",
 ': gram2-opposite': "Антонимы",
 ': gram3-comparative': "Сравнение",
#  ': gram4-superlative',
#  ': gram5-present-participle',
 ': gram6-nationality-adjective': "Национальность - прилагательное",
#  ': gram7-past-tense',
#  ': gram8-plural',
#  ': gram9-plural-verbs'
    }
    df["type"] = "none"
    for i in names:
        if i in df.index:
            df.loc[i, "type"] = names[i]

    return df

In [13]:
def save_table(df, name):
    df.to_csv("./embeddings/{}/tests/{}.csv".format(folder, name))

def get_table( name):
    return pd.read_csv("./embeddings/{}/tests/{}.csv".format(folder, name))

In [ ]:
get_word_m(dct1["росс"], dct1, 10)

{'арас': 0.8767699003219604,
 'ратс': 0.8832137584686279,
 'росс': 1.0,
 'россель': 0.89167720079422,
 'росси': 0.8989118337631226,
 'россий': 0.8773421049118042,
 'росспэн': 0.8937075138092041,
 'росстат': 0.8919171094894409,
 'руссо': 0.887082576751709,
 'ясперс': 0.889279842376709}

##GATS

###generate test

In [14]:
with open("/content/drive/Shareddrives/hse_BERT/hse_Af_Tr_BERT/tests/questions-words.txt", "r") as f:
    lines = f.read().splitlines()

words = {}
words2 = {}
all_w = []
key = "no_key"
for i in lines:
    m = i.split(" ")
    if i[0] == ":":
        key = i
    else:
        if key not in words:
            words[key] = []
            words2[key] = []
        words[key].append(m)
        all_w = all_w + m
        words2[key].append((m[0], m[1]))
        words2[key].append((m[2], m[3]))
all_w = set(all_w)
for key in words2:
    words2[key] = set(words2[key])

In [15]:
names = [
        #  ': capital-common-countries',
 ': capital-world',
#  ': currency',
#  ': city-in-state',
 ': family',
 ': gram1-adjective-to-adverb',
 ': gram2-opposite',
 ': gram3-comparative',
#  ': gram4-superlative',
#  ': gram5-present-participle',
 ': gram6-nationality-adjective',
#  ': gram7-past-tense',
#  ': gram8-plural',
#  ': gram9-plural-verbs'
 ]

In [16]:
words3 = {}
find_words = []
for key in names:
    df = pd.read_csv("tests/gats/{}.csv".format(key[2:]),sep = ";", encoding = "cp1251")
    d1 = list(zip(df["eng_w1"], df["rus_w1"]))
    d2 = list(zip(df["eng_w2"], df["rus_w2"]))
    words3[key] = dict(d1)
    words3[key].update(dict(d2))
    all_rus = [*df["rus_w1"], *df["rus_w2"]]

In [17]:
def translate_sl(m ,sl):
    m2 = []
    for i in m:
        m2.append(sl[i])
    return m2

In [18]:
words_rus = {}
for key in names:
    words_rus[key] = []
    for i in words[key]:
        words_rus[key].append(translate_sl(i ,words3[key]))

In [19]:
with open(os.path.join("embeddings", folder, "tests", "ready.pic"), "bw") as f:
    pc.dump(words_rus, f)

##get quality


In [20]:
with open(os.path.join("embeddings", folder, "tests", "ready.pic"), "br") as f:
    words_rus = pc.load( f)

In [21]:
def cut_b(text):
    # return tokenizer.tokenize("[CLS] " + text.lower() + " [SEP]")[1]
    return text

###3cosadd

In [22]:
def cos_add(key, dct):
    res_df = pd.DataFrame(columns=["w1","w2","w3", "expected", "prdicted", "cosine", "group"])
    
    for m_l in words_rus[key]:
        m = []
        for i in m_l:
            m.append(cut_b(i))
    for m in words_rus[key]:
        if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
            pred, prob = get_word(dct[m[0]] - dct[m[1]] + dct[m[3]], dct, nin = [m[0], m[1], m[3]])
            res_df.loc[res_df.shape[0]] = [m[0], m[1], m[3], m[2], pred, prob, key]
            # print(m[2], pred)
    return res_df

In [23]:
n = 3
def cos_add_3(key, dct):
    res_df = pd.DataFrame(columns=["w1","w2","w3", "expected", "prdicted", "cosine", "group"])
    
    for m_l in words_rus[key]:
        m = []
        for i in m_l:
            m.append(cut_b(i))
    for m in words_rus[key]:
        if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
            pred = get_word_m(dct[m[0]] - dct[m[1]] + dct[m[3]], dct,count = n)
            if m[2] in pred:
                res_df.loc[res_df.shape[0]] = [m[0], m[1], m[3], m[2], m[2], pred[m[2]], key]
            else:
                res_df.loc[res_df.shape[0]] = [m[0], m[1], m[3], m[2], list(pred.keys())[0], pred[list(pred.keys())[0]], key]
            # print(m[2], pred)
    return res_df

In [24]:
for key in words_rus:
    print("start {}".format(key))
    res_df1 = cos_add(key, dct1)
    save_table(res_df1, "3cosadd_res_{}".format(key))
    print("finish {}".format(key))

start : capital-world
finish : capital-world
start : family
finish : family
start : gram1-adjective-to-adverb
finish : gram1-adjective-to-adverb
start : gram2-opposite
finish : gram2-opposite
start : gram3-comparative
finish : gram3-comparative
start : gram6-nationality-adjective
finish : gram6-nationality-adjective


In [36]:
groups = [": capital-world",
": family",
": gram1-adjective-to-adverb",
": gram2-opposite",
": gram3-comparative",
": gram6-nationality-adjective"]

res_df1 = pd.DataFrame()
for key in groups:
    res_df1 = pd.concat([res_df1, get_table("3cosadd_res_{}".format(key))], ignore_index = True )

In [37]:
# get_table("3cosadd_res_{}".format(key))
res_df1

,Unnamed: 0,w1,w2,w3,expected,prdicted,cosine,group
0,0,брат,сестра,мать,отец,отец,0.822723,: family
1,1,отец,мать,сестра,брат,брата,0.873329,: family
2,0,большой,больше,лучше,хорошо,лучший,0.845073,: gram3-comparative
3,1,большой,больше,больше,большой,небольшой,0.947936,: gram3-comparative
4,2,хорошо,лучше,больше,большой,немного,0.853924,: gram3-comparative
5,3,хорошо,лучше,больше,большой,немного,0.853924,: gram3-comparative
6,4,большой,больше,больше,большой,небольшой,0.947936,: gram3-comparative
7,5,большой,больше,лучше,хорошо,лучший,0.845073,: gram3-comparative


In [27]:
# res_df1_3 = cos_add_3(dct1)
# save_table(res_df1_3, "3cosadd_3_res")

for key in words_rus:
    print("start {}".format(key))
    res_df1 = cos_add_3(key, dct1)
    save_table(res_df1, "3cosadd_3_res_{}".format(key))
    print("finish {}".format(key))

start : capital-world
finish : capital-world
start : family
finish : family
start : gram1-adjective-to-adverb
finish : gram1-adjective-to-adverb
start : gram2-opposite
finish : gram2-opposite
start : gram3-comparative
finish : gram3-comparative
start : gram6-nationality-adjective
finish : gram6-nationality-adjective


In [28]:
# res_df1["group"].unique()

res_df1_3 = pd.DataFrame()
for key in groups:
    res_df1_3 = pd.concat([res_df1_3, get_table("3cosadd_3_res_{}".format(key))], ignore_index = True )

In [38]:
res_df1, res_sort_1

(  Unnamed: 0       w1      w2  ...   prdicted    cosine                group
 0          0     брат  сестра  ...       отец  0.822723             : family
 1          1     отец    мать  ...      брата  0.873329             : family
 2          0  большой  больше  ...     лучший  0.845073  : gram3-comparative
 3          1  большой  больше  ...  небольшой  0.947936  : gram3-comparative
 4          2   хорошо   лучше  ...    немного  0.853924  : gram3-comparative
 5          3   хорошо   лучше  ...    немного  0.853924  : gram3-comparative
 6          4  большой  больше  ...  небольшой  0.947936  : gram3-comparative
 7          5  большой  больше  ...     лучший  0.845073  : gram3-comparative
 
 [8 rows x 8 columns], Empty DataFrame
 Columns: [result, count, type]
 Index: [])

In [39]:
res_df1["result"] = res_df1["expected"]  == res_df1["prdicted"] 
count_1 = res_df1.groupby("group").count()
res_sort_1 = res_df1.groupby("group").mean().sort_values(by = "result")
res_sort_1["count"] = count_1["w1"]
res_sort_1 = add_names(res_sort_1)
res_sort_1.index = res_sort_1["type"]
res_sort_1[["cosine", "result", "count"]]

,cosine,result,count
type,,,
Comparative,0.882311,0.0,6
Family,0.848026,0.5,2


In [40]:
save_table(res_sort_1, "3cosadd_itog")

In [41]:
res_df1_3["result"] = res_df1_3["expected"]  == res_df1_3["prdicted"] 
count_1 = res_df1_3.groupby("group").count()
res_3_sort_1 = res_df1_3.groupby("group").mean().sort_values(by = "result")
res_3_sort_1["count"] = count_1["w1"]
res_3_sort_1 = add_names(res_3_sort_1)
res_3_sort_1.index = res_3_sort_1["type"]
res_3_sort_1[["cosine", "result", "count"]]

,cosine,result,count
type,,,
Comparative,0.936110,0.333333,6
Family,0.878964,0.500000,2


In [42]:
save_table(res_3_sort_1, "3cosadd_3_itog")

In [43]:
# cut_b("камбоджа")

###3cosavg

In [44]:
def cos_avg(dct):
    res = pd.DataFrame(columns=["w1","p1","w2","p2","prob", "key","count"])
    for key in words_rus:
        res_df = pd.DataFrame(columns=["w1","w2","w3", "expected", "group"])
        for m_l in words_rus[key]:
            m = []
            for i in m_l:
                
                    m.append(cut_b(i))
                # else:
                #     m.append("none")
        for m in words_rus[key]:
            if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
                res_df.loc[res_df.shape[0]] = [dct[m[0]].numpy(), 
                                               dct[m[1]].numpy(), 
                                               dct[m[3]].numpy(), 
                                               dct[m[2]].numpy(),
                                               key]
                # print(m[2], pred)
            
            
        w1,p1 = get_word(res_df["w1"].mean() - res_df["w2"].mean() + res_df["w3"].mean(), dct)
        w2,p2 = get_word(res_df["expected"].mean(), dct)
        prob = 1 - cosine(res_df["w1"].mean() - res_df["w2"].mean() + res_df["w3"].mean(), res_df["expected"].mean())
        res.loc[res.shape[0]] = [w1,p1,w2,p2, prob, key, res_df.shape[0]]
    return res

In [45]:
res_df1_avg = cos_avg(dct1)
res_df1_avg.index = res_df1_avg["key"]
add_names(res_df1_avg)
res_df1_avg.index = res_df1_avg["type"]
res_df1_avg[["w1", "p1", "w2", "p2", "prob", "count"]]

,w1,p1,w2,p2,prob,count
type,,,,,,
Capital-countries,none,-100,none,-100,NaN,0
Family,отец,0.960428,отец,0.960428,1,2
Adjective adverb,none,-100,none,-100,NaN,0
Opposite,none,-100,none,-100,NaN,0
Comparative,большой,0.981381,большой,0.981381,1,6
Nationality adjective,none,-100,none,-100,NaN,0


In [46]:
save_table(res_df1_avg, "3cosavg_res")

###3cosmul


In [47]:
def get_word_mul(w1,w2,w3, dct):
    word = "none"
    cs = -100 
    for key in dct:
        e1 = 1 - cosine(dct[w1], dct[key])
        e2 = 1 - cosine(dct[w2], dct[key])
        e3 = 1 - cosine(dct[w3], dct[key])
        if e1 - e2 + e3 > cs and key != w1 and key != w2 and key != w3:
            cs = e1 - e2 + e3
            word = key
    return word, cs

In [48]:
def cos_mul(key, dct):
    res_df = pd.DataFrame(columns=["w1","w2","w3", "expected", "prdicted", "cosine", "group"])
    for m_l in words_rus[key]:
        m = []
        for i in m_l:
            
            m.append(cut_b(i))
            # else:
            #     m.append("none")
    for m in words_rus[key]:
        if m[0] in dct and m[1] in dct and m[2] in dct and m[3] in dct:
            pred, prob = get_word_mul(m[0], m[1], m[3], dct)
            res_df.loc[res_df.shape[0]] = [m[0], m[1], m[3], m[2], pred, prob, key]
                # print(m[2], pred)
    return res_df

In [49]:
for key in words_rus:
    print("start {}".format(key))
    res_df1 = cos_mul(key, dct1)
    save_table(res_df1, "3cosmul_res_{}".format(key))
    print("finish {}".format(key))

start : capital-world
finish : capital-world
start : family
finish : family
start : gram1-adjective-to-adverb
finish : gram1-adjective-to-adverb
start : gram2-opposite
finish : gram2-opposite
start : gram3-comparative
finish : gram3-comparative
start : gram6-nationality-adjective
finish : gram6-nationality-adjective


In [50]:
groups = [": capital-world",
": family",
": gram1-adjective-to-adverb",
": gram2-opposite",
": gram3-comparative",
": gram6-nationality-adjective"]

res_df1_mul = pd.DataFrame()
for key in groups:
    res_df1_mul = pd.concat([res_df1_mul, get_table("3cosmul_res_{}".format(key))], ignore_index = True )

In [51]:
# res_df1_mul = cos_mul(dct1)

# save_table(res_df1_mul, "3cosmul_res")
res_df1_mul["result"] = res_df1_mul["expected"]  == res_df1_mul["prdicted"] 
count_1 = res_df1_mul.groupby("group").count()
res_df_1_mul = res_df1_mul.groupby("group").mean().sort_values(by = "result")
res_df_1_mul["count"] = count_1["w1"]
res_df_1_mul = add_names(res_df_1_mul)
res_df_1_mul.index = res_df_1_mul["type"]
res_df_1_mul[["cosine", "result", "count"]]
save_table(res_df_1_mul, "3cosmul_itog")

In [52]:
save_table(res_df1_mul, "3cosmul_res")
save_table(res_df_1_mul, "3cosmul_itog")

###mix table

In [ ]:
ddd1 = pd.DataFrame()
def get_r1(r1, metric):
    r1["metric"] = metric
    return r1.pivot(columns = ["metric"])

In [ ]:
ddd1 = get_r1(res_sort_1[["cosine", "result", "count"]], "3CosAdd")
ddd1 = ddd1.merge(get_r1(res_df_1_mul[["cosine", "result", "count"]], "3CosMul"), on = "type")
r1 = res_df1_avg[["prob", "count"]].copy()
r1["cosine"] = r1["prob"]
ddd1 = ddd1.merge(get_r1(r1[["cosine", "count"]], "3CosAvg"), on = "type")
ddd1

,cosine,result,count,cosine,result,count,cosine,count
metric,3CosAdd,3CosAdd,3CosAdd,3CosMul,3CosMul,3CosMul,3CosAvg,3CosAvg
type,,,,,,,,
Adjective adverb,0.877269,0.306324,506,0.960834,0.306324,506,1.000000,506
Capital-countries,0.804135,0.000000,115,0.916737,0.000000,115,0.996707,115
Comparative,0.877970,0.370370,756,0.958734,0.366402,756,1.000000,756
Family,0.883500,0.384615,156,0.950992,0.378205,156,1.000000,156
Nationality adjective,0.873831,0.367470,332,0.932075,0.361446,332,0.999993,332
Opposite,0.851062,0.000000,30,0.926910,0.000000,30,1.000000,30


In [ ]:
ddd1[["result", "cosine", "count"]]

result              cosine  ...   count                
metric                  3CosAdd   3CosMul   3CosAdd  ... 3CosAdd 3CosMul 3CosAvg
type                                                 ...                        
Adjective adverb       0.306324  0.306324  0.877269  ...     506     506     506
Capital-countries      0.000000  0.000000  0.804135  ...     115     115     115
Comparative            0.370370  0.366402  0.877970  ...     756     756     756
Family                 0.384615  0.378205  0.883500  ...     156     156     156
Nationality adjective  0.367470  0.361446  0.873831  ...     332     332     332
Opposite               0.000000  0.000000  0.851062  ...      30      30      30

[6 rows x 8 columns]